In [3]:
# echo-server.py
import selectors
import socket
from _thread import *
import threading
import time
from random import randint
import json 

LAN_IP = "192.168.1.255"  # Standard loopback interface address (localhost)
PORT = 9999  # Port to listen on (non-privileged ports are > 1023)

In [33]:
def parse_data(data):
    msg_type = None
    event = None
    sender_id = None
    relay = 0
    
    return msg_type


def thread(Client, Pserver):
    while True:
        # data received from client
        data = Pserver.recv(1024)
        if data:
            Client.send(data)
        elif not data:
            print_lock.release()
            break


class Vehicle:
    def __init__(self, LAN_IP="127.0.0.1", PORT=9999):
        self.location = 0 # 1 dimensional x coordinate
        self.last_update = time.time()
        self.msg = ""
        self.buffer = ""
        self.proximity_table = []
        self.velocity = 1 # m/s
        self.LAN_IP = LAN_IP
        self.PORT = PORT
        self.id = randint(0, 1000)
        
        #self.broadcast = 
        try:
            self.rx = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.tx = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.rx.bind((LAN_IP, PORT))
        except Exception as e:
            print(e) 
    

        
    def update_location(self):
        current_time = time.time()
        self.location = self.location + self.velocity * (current_time - self.last_update)
        self.last_update = current_time
        return self.location


    def broadcast_location(self):
        try:
            self.update_location()

            broadcast_loc_msg = {"msg_type": "location_update", "time": time.time(), "vehicle.id": self.id, 
            "location": str(self.location)}

            self.tx.sendall(json.dumps(broadcast_loc_msg).encode())
        except Exception as e:
            print(e) 
        return 1
        

    def send_event_msg(self, msg):
        try:
            self.tx.sendall(msg)
            print("request sent")
        except Exception as e:
            print(e)
        
        # read acks as they come in for 2s and then 
        # resend broadcast notification w updated 
        
        return 1
    
    
    def recieve(self):
        try:
            self.recieve_data = self.rx.recv(4096)
            if self.recieve_data:
                print("recieved reply")
                return self.recieve_data
            elif not self.recieve_data:
                return None
        except Exception as e:
            print(e)



def emergency_event(vehicle):
    ack_table = vehicle.proximity_table.copy()
    time = time.time()
    while ack_table:
        emergency_msg = {"msg_type": "crash", "time": time, "vehicle.id": vehicle.id, 
            "proximity_table": ack_table}
        vehicle.send_event_msg(json.dumps(emergency_event).encode())

        data = vehicle.recieve()

        if data:


    return True

    
    

In [19]:
MESSAGE = b"Hello, World!"
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)

In [17]:
sock.sendto(MESSAGE, (LAN_IP, PORT))

13

In [21]:
sock.bind((LAN_IP, PORT))
while True:
    data, addr = sock.recvfrom(1024)
    print("recieved msg %s" %data)

recieved msg b'Hello, World!'
recieved msg b'Hello, World!'
recieved msg b'Hello, World!'
recieved msg b'Hello, World!'
recieved msg b'Hello, World! from tm1'


In [11]:
randint(0,1000) > 500

True

In [ ]:
if __name__ == "_main__":
    vehicle = Vehicle(LAN_IP= LAN_IP)
    
    while True:
        vehicle.broadcast()

        vehicle.recieve()
        if vehicle.recieve_data:
            #parse data
            
            #-------------------if else event/recieve statements-----------------
            #if is broadcast_location: 
            #     if close: update proximity table

            #else is event:
                #if parse for ID: send ack
        
        #Initiate emergency message
        #if randint(0,1000) > 500:
        
            break
        


In [40]:
vehicle = Vehicle()

In [41]:
vehicle.proximity_table

[]

In [42]:
tmp = vehicle.proximity_table.copy()

In [66]:
tmp.append("test")

In [67]:
tmp

['test1', 'test']

In [45]:
vehicle.proximity_table


[]

In [65]:
tmp.remove("test")

ValueError: list.remove(x): x not in list

In [68]:
"test" in tmp

True